In [3]:
import sys
import pandas as pd
sys.path.append('../src')
from restart import NoteCompose
from util import set_config, to_df, to_sheet, display_population, format_population

config = set_config('../src')
restart = NoteCompose(configdir='../src/', population='dict')
population = restart.model.population
resource = restart.model.resource
demand = restart.model.demand
inventory=restart.model.inventory

In [12]:
# read IHME data into dataframe
dfihmen = pd.read_csv('../../data/epidemiological/global/IHME/Reference_Hospitalization_All_Locs.csv')

# filter dataframe for California only
dfihmen = dfihmen.loc[dfihmen['location_name'] == 'California']

# Find maximum forecasted number of covid patients needing ICU beds 
max_icuIHME = (dfihmen.loc[dfihmen['ICUbed_upper'] == max(dfihmen['ICUbed_upper'])].reset_index())['ICUbed_upper'][0]
max_bedIHME = (dfihmen.loc[dfihmen['allbed_upper'] == max(dfihmen['allbed_upper'])].reset_index())['allbed_upper'][0]
max_ventIHME = (dfihmen.loc[dfihmen['InvVen_upper'] == max(dfihmen['InvVen_upper'])].reset_index())['InvVen_upper'][0]
# print('IHME Max COVID Patients Needing ICU Bed is ' + str(max_icuIHME))
# print('IHME Max COVID Patients Needing Hospital Bed is ' +str(max_bedIHME))
# print('IHME Max COVID Patients Needing Ventilation is ' + str(max_ventIHME))

# read MIT CA data into dataframe
dfmit = pd.read_csv('../../data/epidemiological/us/forecasts/MIT/state/hospital/MIT_state_CA_hospitalproj.csv')

# Find maximum forecasted number of active ventilated and hospitalized covid patients
max_ventMIT = dfmit.loc[dfmit['Active Ventilated'] == max(dfmit['Active Ventilated'])]['Active Ventilated'][0]
max_hospMIT = dfmit.loc[dfmit['Active Hospitalized'] == max(dfmit['Active Hospitalized'])]['Active Hospitalized'][0]
# print('MIT Max COVID Patients Active Hospitalized is ' + str(max_hospMIT))
# print('MIT Max COVID Patients Needing Ventilation is ' + str(max_ventMIT))

In [5]:
pop_sheet = format_population(to_sheet(population.population_pP_tr.df), round=True)
daily_sheet = format_population(to_sheet(demand.demand_by_pop_total_pn_tc.df), round=True)
burn_sheet = format_population(to_sheet(demand.demand_per_unit_map_dn_um.df))

In [11]:
import ipyvuetify as v
v.theme.themes.light.primary = 'colors.teal'

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['Home']),
    v.Tab(children=['Projected Patients']),
    v.Tab(children=['Daily Demand']),
    v.Tab(children=['Burn Rates']),
    v.TabItem(children = [
        v.Html(tag='h1', children=['Restart Partners Projection']),
        v.Html(tag='p', children=[
            "We project that a purchase of additional ventilators is not necessary. In our stockpile analysis we used several different models, including the CHA PPE projections, IHME COVID surge predictions, and MIT COVID surge predictions. The maximum projected ventilator need we found was 8,289, based on the IHME surge predictions."
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Population"]),
                v.CardSubtitle(children=["Projected Patients"]),
                pop_sheet
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Demand"]),
                v.CardSubtitle(children=["Daily Resource Demand"]),
                daily_sheet
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Burn Rates"]),
                v.CardSubtitle(children=["Per Capita Resource Demand"]),
                burn_sheet
            ])
        ])
    ])
])

Tabs(children=[Tab(children=['Home']), Tab(children=['Projected Patients']), Tab(children=['Daily Demand']), T…